In [1]:
import tensorflow as tf
import numpy as np

# Preprocessing

## Read image into tf

In [2]:
import os.path

In [3]:
IMAGE_DIR = os.path.abspath('../../data/png/')
IMAGE_FILE_WILDCARD = '/*.png'

In [4]:
# from glob import glob
# filenames = glob(IMAGE_DIR + IMAGE_FILE_WILDCARD)

In [5]:
# Make a queue of file names including all the images files in the relative
# image directory.
filenames = tf.train.match_filenames_once(IMAGE_DIR + IMAGE_FILE_WILDCARD)
filename_queue = tf.train.string_input_producer(filenames)

In [6]:
image_reader = tf.WholeFileReader()

In [7]:
# Read a whole file from the queue
filename, image_file = image_reader.read(filename_queue)

In [19]:
# Decode the image as a PNG file, this will turn it into a Tensor which we can
# then use in training.
image = tf.image.decode_png(image_file, channels=3)

In [31]:
# Start a new session to show example output.
with tf.Session() as sess:
    # Required to get the filename matching to run.
    # lesson learned... https://stackoverflow.com/questions/44143139/tensorflow-operation-tf-train-match-filenames-once-not-working
    init = (tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init)

    # Coordinate the loading of image files.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    try:
        while True:
            # Get an image tensor and print its value.
            filename_tensor, image_tensor = sess.run([filename, image])
            print(image_tensor.shape)
            print(filename_tensor)
            
            coord.request_stop()
            coord.join(threads)
    except tf.errors.OutOfRangeError:
        # This will be raised when you reach the end of an epoch (i.e. the
        # iterator has no more elements).
        print('no more elements')

    # Perform any end-of-epoch computation here.
    print('Done training')



(1684, 1190, 3)
b'/Users/ericng/Workspace/doc-pic-classifier/data/png/doc_template_01.1506756435.2.html.png-clipped.png'
(1684, 1190, 3)
b'/Users/ericng/Workspace/doc-pic-classifier/data/png/doc_template_02.1506756693.8.html.png-clipped.png'
(1684, 1190, 3)
b'/Users/ericng/Workspace/doc-pic-classifier/data/png/doc_template_01.1506756693.7.html.png-clipped.png'
(1684, 1190, 3)
b'/Users/ericng/Workspace/doc-pic-classifier/data/png/doc_template_01.1506750812.4.html.png-clipped.png'
(1684, 1190, 3)
b'/Users/ericng/Workspace/doc-pic-classifier/data/png/doc_template_01.1506750812.8.html.png-clipped.png'
(1684, 1190, 3)
b'/Users/ericng/Workspace/doc-pic-classifier/data/png/doc_template_01.1506756435.4.html.png-clipped.png'
(1684, 1190, 3)
b'/Users/ericng/Workspace/doc-pic-classifier/data/png/doc_template_01.1506756693.9.html.png-clipped.png'
(1684, 1190, 3)
b'/Users/ericng/Workspace/doc-pic-classifier/data/png/doc_template_01.1506750812.2.html.png-clipped.png'
(1684, 1190, 3)
b'/Users/ericng/